In [1]:
import pandas as pd
from unidecode import unidecode

In [2]:
def ClassificaProfissao(profissao, arquivoComProfissoes):
    if profissao == "ESTUDANTE":
        return "ESTUDANTE"
    elif profissao in arquivoComProfissoes['Grupo 1'].values:
        return "GRUPO 1"
    elif profissao in arquivoComProfissoes['Grupo 2'].values:
        return "GRUPO 2"
    elif profissao in arquivoComProfissoes['Grupo 3'].values:
        return "GRUPO 3"
    elif profissao in arquivoComProfissoes['Grupo 4'].values:
        return "GRUPO 4"
    elif profissao in arquivoComProfissoes['Grupo 5'].values:
        return "GRUPO 5"
    else: 
        return "OUTROS"
    
def PadronizaMunicipio(municipio):
    if isinstance(municipio, str):
        municipioLimpo = municipio.split('|')[-1].strip()
        return municipioLimpo
    return municipio

def PadronizaPais(pais):
    if isinstance(pais, str) and '|' in pais:
        paisLimpo = pais.split('|')[-1].strip()
        return paisLimpo
    return pais

In [ ]:
# Lê o novo arquivo
novoArquivo = "Inserir aqui o nome do novo arquivo"
novaTabela = pd.read_csv(novoArquivo, dtype=str)
print("Novo arquivo lido")

# Lê o arquivo com o nome das colunas a serem subistituidas
tabelaComNomesDasColunas = pd.read_excel("Comparação dos campos de cada tabela.xlsx")

# Verifica se a nova tabela manteve o mesmo padrão
padraoAtual = tabelaComNomesDasColunas.iloc[16].tolist()
padraoAtual.pop(0)
padraoNaNovaTabela = novaTabela.iloc[0].tolist()

if not set(padraoAtual) == set(padraoNaNovaTabela):
    print("A nova tabela mudou o padrão no dados. As seguintes colunas não batem:")

    valoresUnicosNaTabelaAtual = set(padraoAtual) - set(padraoNaNovaTabela)
    valoresUnicosTabelaAntiga = set(padraoNaNovaTabela) - set(padraoAtual)

    if valoresUnicosNaTabelaAtual:
        print("Valores na tabela antiga que não estão na nova tabela:", valoresUnicosNaTabelaAtual)
    if valoresUnicosTabelaAntiga:
        print("Valores na nova tabela que não estão na tabela antiga:", valoresUnicosTabelaAntiga)
else:
    print("As tabelas possuem os mesmos valores nas colunas respectivas.")
    padraoNaNovaTabela = [item for item in padraoNaNovaTabela if not pd.isna(item)]
    novaTabela.columns = tabelaComNomesDasColunas.iloc[14]

# Lê o arquivo antigo
antigaTabela = pd.read_excel('2014 a 2023-2.xlsx', dtype=str)
print("Antigo arquivo lido")

# Lê o arquivo com a lista de municípios do Brasil
arquivoComMunicipios = pd.read_csv("municipios.csv", dtype=str, delimiter=';', encoding = "ISO-8859-1")
arquivoComMunicipios['MUNICÍPIO - TOM'] = arquivoComMunicipios['MUNICÍPIO - TOM'].apply(unidecode)

# Lê o arquivo com as categorias de profissão do ENEM
arquivoComProfissoes = pd.read_excel("profissoes.xlsx", dtype=str)
arquivoComProfissoes = arquivoComProfissoes.applymap(lambda x: x.upper() if isinstance(x, str) else x)

# Limpezas especificas
novaTabela = novaTabela.drop_duplicates(subset='Hash', keep='first')
for coluna in novaTabela.columns: novaTabela[coluna] = novaTabela[coluna].replace(['NAO SABE INFORMAR', 'N/D', "ATENDIMENTO INTERROMPIDO", "DENUNCIANTE NAO SOUBE INFORMAR", "NAO INFORMADO"], '')
novaTabela = novaTabela.map(lambda x: x.strip() if isinstance(x, str) else x)

################################################################

novaTabela['País'] = novaTabela['País'].apply(PadronizaPais)

novaTabela['UF'] = novaTabela['UF'].str.strip()
novaTabela['UF'] = novaTabela['UF'].mask(~novaTabela['UF'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['Município'] = novaTabela['Município'].apply(PadronizaMunicipio)
novaTabela['Município'] = novaTabela['Município'].where(novaTabela['Município'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

novaTabela['Nacionalidade da vítima'] = novaTabela['Nacionalidade da vítima'].apply(PadronizaPais)

novaTabela['País da vítima'] = novaTabela['País da vítima'].apply(PadronizaPais)

novaTabela['UF da vítima'] = novaTabela['UF da vítima'].str.strip()
novaTabela['UF da vítima'] = novaTabela['UF da vítima'].mask(~novaTabela['UF da vítima'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['Município da vítima'] = novaTabela['Município da vítima'].apply(PadronizaMunicipio)
novaTabela['Município da vítima'] = novaTabela['Município da vítima'].where(novaTabela['Município da vítima'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

novaTabela['sl vitima naturalizado uf'] = novaTabela['sl vitima naturalizado uf'].str.strip()
novaTabela['sl vitima naturalizado uf'] = novaTabela['sl vitima naturalizado uf'].mask(~novaTabela['sl vitima naturalizado uf'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['sl vitima naturalizado municipio'] = novaTabela['sl vitima naturalizado municipio'].apply(PadronizaMunicipio)
novaTabela['sl vitima naturalizado municipio'] = novaTabela['sl vitima naturalizado municipio'].where(novaTabela['sl vitima naturalizado municipio'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

novaTabela['Suspeito nacionalidade'] = novaTabela['Nacionalidade da vítima'].apply(PadronizaPais)

novaTabela['País do suspeito'] = novaTabela['País do suspeito'].apply(PadronizaPais)

novaTabela['UF do suspeito'] = novaTabela['UF do suspeito'].str.strip()
novaTabela['UF do suspeito'] = novaTabela['UF do suspeito'].mask(~novaTabela['UF do suspeito'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['sl suspeito municipio'] = novaTabela['sl suspeito municipio'].apply(PadronizaMunicipio)
novaTabela['sl suspeito municipio'] = novaTabela['sl suspeito municipio'].where(novaTabela['sl suspeito municipio'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

novaTabela['sl suspeito naturalizado uf'] = novaTabela['sl suspeito naturalizado uf'].str.strip()
novaTabela['sl suspeito naturalizado uf'] = novaTabela['sl suspeito naturalizado uf'].mask(~novaTabela['sl suspeito naturalizado uf'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

novaTabela['sl suspeito naturalizado municipio'] = novaTabela['sl suspeito naturalizado municipio'].apply(PadronizaMunicipio)
novaTabela['sl suspeito naturalizado municipio'] = novaTabela['sl suspeito naturalizado municipio'].where(novaTabela['sl suspeito naturalizado municipio'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

novaTabela['Sexo da vítima'] = novaTabela['Sexo da vítima'].mask(~novaTabela['Sexo da vítima'].isin(['FEMININO', 'MASCULINO']), '')
novaTabela['Sexo do suspeito'] = novaTabela['Sexo do suspeito'].mask(~novaTabela['Sexo do suspeito'].isin(['FEMININO', 'MASCULINO']), '')

novaTabela['Faixa etária da vítima'] = novaTabela['Faixa etária da vítima'].replace(['10 ANOS', '11 ANOS'], '10 A 11 ANOS')
novaTabela['Faixa etária da vítima'] = novaTabela['Faixa etária da vítima'].replace(['12 ANOS', '13 ANOS', '14 ANOS'], '12 A 14 ANOS')
novaTabela['Faixa etária do suspeito'] = novaTabela['Faixa etária do suspeito'].replace(['15 ANOS', '16 ANOS', '17 ANOS'], '15 A 17 ANOS')
novaTabela['Faixa etária do suspeito'] = novaTabela['Faixa etária do suspeito'].replace(['12 ANOS', '13 ANOS', '14 ANOS'], '12 A 14 ANOS')

novaTabela['Deficiência da vítima'] = novaTabela['Deficiência da vítima'].replace(['NAO'], 'NAO TEM DEFICIENCIA')
novaTabela['Deficiência do suspeito'] = novaTabela['Deficiência do suspeito'].replace(['NAO'], 'NAO TEM DEFICIENCIA')

novaTabela['Doença rara da vítima'] = novaTabela['Doença rara do suspeito'].replace(['NAO'], 'NAO TEM DOENCA RARA')
novaTabela['Doença rara do suspeito'] = novaTabela['Doença rara do suspeito'].replace(['NAO'], 'NAO TEM DOENCA RARA')

novaTabela['sl vitima cadastro'] = novaTabela['sl vitima cadastro'].replace(['COMUNIDADE', 'FAMILIA'], 'COMUNIDADE/FAMILIA')
novaTabela['Natureza Jurídica do Suspeito'] = novaTabela['Natureza Jurídica do Suspeito'].replace(['COMUNIDADE', 'FAMILIA'], 'COMUNIDADE/FAMILIA')

novaTabela['Orientação sexual da vítima'] = novaTabela['Orientação sexual da vítima'].replace(['HOMOSSEXUAL.LESBICA', 'HOMOSSEXUAL.GAY'], 'HOMOSSEXUAL')
novaTabela['Orientação sexual do suspeito'] = novaTabela['Orientação sexual do suspeito'].replace(['HOMOSSEXUAL.LESBICA', 'HOMOSSEXUAL.GAY'], 'HOMOSSEXUAL')

novaTabela['Grau de instrução da vítima'] = novaTabela['Grau de instrução da vítima'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')
novaTabela['Grau de instrução do suspeito'] = novaTabela['Grau de instrução do suspeito'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')

novaTabela['Religião da vítima'] = novaTabela['Religião da vítima'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')
novaTabela['Religião do suspeito'] = novaTabela['Religião do suspeito'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')

novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['PRETO', 'PARDO', 'PARDA'], '')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['COMUNIDADES QUILOMBOLAS'], 'QUILOMBOLAS')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['POVOS INDIGENAS'], 'INDIGENA')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['DEMAIS COMUNIDADES TRADICIONAIS'], 'POVOS E COMUNIDADES TRADICIONAIS')
novaTabela['Etnia da vítima'] = novaTabela['Etnia da vítima'].replace(['POVOS CIGANOS'], 'CIGANA')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['PRETO', 'PARDO', 'PARDA'], '')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['COMUNIDADES QUILOMBOLAS'], 'QUILOMBOLAS')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['POVOS INDIGENAS'], 'INDIGENA')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['DEMAIS COMUNIDADES TRADICIONAIS'], 'POVOS E COMUNIDADES TRADICIONAIS')
novaTabela['Etnia do Suspeito'] = novaTabela['Etnia do Suspeito'].replace(['POVOS CIGANOS'], 'CIGANO')

novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['ALUNO (A)'], 'ALUNO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['AMIGO (A)'], 'AMIGO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['AVO(O)'], 'AVO')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['COMPANHEIRO (A)'], 'COMPANHEIRO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['CUIDADOR (A)'], 'CUIDADOR(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['CUNHADO (A)'], 'CUNHADO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EMPREGADO(HIERARQUICAMENTE INFERIOR)'], 'EMPREGADO (HIERARQUICAMENTE INFERIOR)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EMPREGADOR/ PATRAO(HIERARQUICAMENTE SUPERIOR)', 'EMPREGADOR/PATRAO(HIERARQUICAMENTE SUPERIOR)'], 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['ESPOSA', 'MARIDO'], 'ESPOSO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EX-COMPANHEIRO (A)'], 'EX-COMPANHEIRO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EX-ESPOSA', 'EX-MARIDO'], 'EX-ESPOSO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['EX-NAMORADO (A)'], 'EX-NAMORADO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['FILHO (A)'], 'FILHO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['IRMAO (A)'], 'IRMAO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['LIDER RELIGIOSO'], 'LIDER RELIGIOSO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['MORA NA MESMA RESIDENCIA MAS NAO E FAMILIAR', 'MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR'], 'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['FAMILIARES', 'OUTROS FAMILIARES', 'OUTROS PROFISSIONAIS DA EDUCACAO'], 'OUTROS')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['PRESTADOR DE SERVICO'], 'PRESTADOR(A) DE SERVICO')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['TIO (A)'], 'TIO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['TREINADOR/TECNICO'], 'TREINADOR(A)/TECNICO(A)')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['TRISAVO(O)'], 'TRISAVO')
novaTabela['Relação vítima-suspeito'] = novaTabela['Relação vítima-suspeito'].replace(['VIZINHO (A)'], 'VIZINHO(A)')

novaTabela['Violações'] = novaTabela['Violações'].str.replace(';', '', regex=False)

substituicoes = {
    '1 A 3 SM': '1 A 3 SM',
    '3 A 5 SM': '3 A 5 SM',
    '5 A 15 SM': '5 A 15 SM',
    '6 A 15 SM': '5 A 15 SM',
    '8 A 15 SM': '5 A 15 SM',
    'ACIMA DE 15 SM': 'ACIMA DE 15 SM',
    'ATE 1 SM': 'ATE 1 SM',
    'ATE 1/2 SALARIO MINIMO': 'ATE 1/2 SM',
    'MAIS DE 1 A 2 SALARIOS MINIMOS': '1 A 3 SM',
    'MAIS DE 1/2 A 1 SALARIO MINIMO': 'ATE 1 SM',
    'MAIS DE 10 A 20 SALARIOS MINIMOS': '10 A 20 SM',
    'MAIS DE 2 A 5 SALARIOS MINIMOS': '3 A 5 SM',
    'MAIS DE 20 SALARIOS MINIMOS': 'MAIS DE 20 SM',
    'MAIS DE 5 A 10 SALARIOS MINIMOS': '5 A 15 SM',
    'SEM RENDIMENTO': 'SEM RENDIMENTO'
}

novaTabela['Faixa de renda da vítima'] = novaTabela['Faixa de renda da vítima'].replace(substituicoes)

novaTabela = novaTabela.drop('Hash', axis=1)
novaTabela = novaTabela.drop('Grupo vulnerável', axis=1)

# Padroniza conforme dicionario de dados do ENEM
novaTabela['Profissão da vítima'] = novaTabela['Profissão da vítima'].apply(lambda profissao: ClassificaProfissao(profissao, arquivoComProfissoes))
novaTabela['Profissão do suspeito'] = novaTabela['Profissão do suspeito'].apply(lambda profissao: ClassificaProfissao(profissao, arquivoComProfissoes))
print("Nova tabela limpa")

# Inseri os novos dados
tabelaInserida = pd.concat([novaTabela, antigaTabela], ignore_index=True)
print("Nova tabela anexado ao dados")

# Salva a tabela unificada
tabelaInserida.to_excel("../Etapa 3/Dados Atualizados.xlsx", index=False)
print("Novo arquivo salvo\n")